## Classificação

In [1]:
#imports
import numpy as np
from sklearn import svm, metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, RFE, f_classif

Load

In [2]:
features = np.load("processed/extracted_features.npy").swapaxes(1,2)
features.shape

(2, 224, 18)

In [3]:
# protocolo
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

# # criação do vetor de labels final
# y = labels_num * int(features.shape[1] / len(labels_num))
# print(y)
# len(y)

y = np.repeat(labels_num, int(features.shape[1] / len(labels_num)), axis=None)
print(y)
len(y)

[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 2 2 2 2 2
 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 3 3
 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 2 2 2 2
 2 2 2 2 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 3 3 3 3 3 3
 3 3]


224

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [4]:
# aplicando normalização

for i in range(len(features)):
  features[i] = StandardScaler().fit_transform(features[i])

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

In [5]:
# aplicando seleção de características
X_all = list()

threshold = .1
k = 9
n_components = 6


for i in range(len(features)):
  print(f'Participante {i} ------- ')
  
  # ORIGINAL
  X_all.append((f'p{i}-original', features[i]))
  print(features[i].shape)

  # SELECT K BEST
  kb = SelectKBest(f_classif, k=k).fit_transform(features[i], y)
  X_all.append((f'p{i}-kBest', kb))
  print(kb.shape)

  # VARIANCE THRESHOLD
  vt = VarianceThreshold(threshold).fit_transform(features[i])
  X_all.append((f'p{i}-vTresh', vt))
  print(vt.shape)

  # RECURSIVE FEATURE ELIMINATION
  rfe = RFE(svm.SVC(kernel="linear")).fit_transform(features[i], y)
  X_all.append((f'p{i}-RFE', rfe))
  print(rfe.shape)

  # # PRINCIPAL COMPONENT ANALISYS
  # pca = PCA(n_components).fit_transform(features[i])
  # X_all.append((f'p{i}-PCA', pca))
  # print(pca.shape)

Participante 0 ------- 
(224, 18)
(224, 9)
(224, 16)
(224, 9)
Participante 1 ------- 
(224, 18)
(224, 9)
(224, 16)


/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0 1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0 1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


(224, 9)


*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [6]:
# aplicando a classificação

for X in X_all:
    print(f'{X[0]} --------------------------------------------')
    # Divisão Treino/Teste
    x_train, x_test, y_train, y_test = train_test_split(X[1], y, test_size=0.25, random_state=42)
    print('Treino >', x_train.shape, len(y_train))
    print('Teste >', x_test.shape, len(y_test))
    
    # Grid search
    # param_grid = {
    #     'C': [0.1, 1, 10, 100],
    #     'kernel': ['linear', 'rbf', 'poly'],#, 'sigmoid'],
    #     'gamma': [0.01, 0.1, 1, 10],
    #     # 'degree': [2, 3, 4],
    #     # 'coef0': [0, 0.5, 1],
    #     # 'tol': [1e-4, 1e-5, 1e-6]
    # }
   
    param_grid = [
        {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
        {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'rbf', 'poly']},
    ]

    grid = GridSearchCV(svm.SVC(), param_grid, cv=5)
    grid.fit(x_train, y_train)

    # Predição
    y_pred = grid.predict(x_test)
    print('Acurácia >', metrics.accuracy_score(y_test, y_pred), '\n')
    # print(metrics.classification_report(y_test, y_pred, digits=5))

p0-original --------------------------------------------
Treino > (168, 18) 168
Teste > (56, 18) 56


Acurácia > 0.4642857142857143 

p0-kBest --------------------------------------------
Treino > (168, 9) 168
Teste > (56, 9) 56
Acurácia > 0.5 

p0-vTresh --------------------------------------------
Treino > (168, 16) 168
Teste > (56, 16) 56
Acurácia > 0.4642857142857143 

p0-RFE --------------------------------------------
Treino > (168, 9) 168
Teste > (56, 9) 56
Acurácia > 0.6785714285714286 

p1-original --------------------------------------------
Treino > (168, 18) 168
Teste > (56, 18) 56
Acurácia > 0.39285714285714285 

p1-kBest --------------------------------------------
Treino > (168, 9) 168
Teste > (56, 9) 56
Acurácia > 0.30357142857142855 

p1-vTresh --------------------------------------------
Treino > (168, 16) 168
Teste > (56, 16) 56
Acurácia > 0.39285714285714285 

p1-RFE --------------------------------------------
Treino > (168, 9) 168
Teste > (56, 9) 56
Acurácia > 0.48214285714285715 

